# Analyze results of Timeloop/Accelergy on NeRF - Dot Product

In [21]:
import json
import os
import pandas as pd
import regex as re

In [22]:
!ls profile_results

dot-product_results.json
eyeriss_like_onchip_compression_results.json
eyeriss_like_results.json
eyeriss_like_shen_results.json
eyeriss_like_w_gating_results.json
results.json
simba_like_results.json
simple_weight_stationary_results.json


## Load Results from the Profile notebok

In [23]:
def load_results(results_path: str = "profile_results/dot-product_results.json") -> dict:
    """ Load results from process_dot-product_results.py back into original shape. """
    with open(results_path, "r") as f:
        results = json.load(f)
    print(f"Loaded results from {results_path}")
        
    return results

profile_results = load_results()

Loaded results from profile_results/dot-product_results.json


In [24]:
profile_results

{'dot-product-DUDU': {'gflops': 1.99,
  'utilization': 1.0,
  'cycle': 128,
  'energy': 0.0,
  'edp': 2.87e-07,
  'area': 0.0,
  'total_energy': 10460.09,
  'total_area': 2243.6099999999997},
 'dot-product-SCDU': {'gflops': 1.98,
  'utilization': 1.0,
  'cycle': 48,
  'energy': 0.0,
  'edp': 1.29e-07,
  'area': 0.0,
  'total_energy': 33892.08,
  'total_area': 2686.0600000000004},
 'dot-product-SUDU': {'gflops': 1.99,
  'utilization': 1.0,
  'cycle': 128,
  'energy': 0.0,
  'edp': 2.87e-07,
  'area': 0.0,
  'total_energy': 10460.09,
  'total_area': 2243.6099999999997},
 'dot-product-SUDU_no-optimization': {'total topology energy': 35.44,
  'total topology area': 1220.09,
  'max topology cycles': 16,
  'utilization': 1.0,
  'cycle': 16,
  'energy': 0.0,
  'area': 0.0,
  'total_energy': 1220.0900000000001,
  'total_area': 35.44},
 'dot-product-SUDU_gating': {'total topology energy': 13.75,
  'total topology area': 1220.09,
  'max topology cycles': 16,
  'utilization': 0.25,
  'cycle': 16,

## Analyze

In [25]:
# Massage results into dataframes
layer_dfs = {}
all_summary = {}
# Remove area and total_area as they are wrong as just 0.0
drop_cols = ["name", "stats_fname", "total_area", "area"]

for arch, arch_results in profile_results.items():
    all_summary[arch] = arch_results

    # Load layer results into dataframe
    df_layer = pd.DataFrame.from_dict(arch_results, orient="index")
    # df_layer = df_layer.drop(columns=drop_cols)
    df_layer.index.name = arch
    layer_dfs[arch] = df_layer
    
df_summary = pd.DataFrame.from_dict(all_summary, orient="index")
# df_summary = df_summary.drop(columns=["total_area"])

In [26]:
df_summary

,gflops,utilization,cycle,energy,edp,area,total_energy,total_area,total topology energy,total topology area,max topology cycles
dot-product-DUDU,1.99,1.00,128,0.0,2.870000e-07,0.0,10460.09,2243.61,NaN,NaN,NaN
dot-product-SCDU,1.98,1.00,48,0.0,1.290000e-07,0.0,33892.08,2686.06,NaN,NaN,NaN
dot-product-SUDU,1.99,1.00,128,0.0,2.870000e-07,0.0,10460.09,2243.61,NaN,NaN,NaN
dot-product-SUDU_no-optimization,NaN,1.00,16,0.0,NaN,0.0,1220.09,35.44,35.44,1220.09,16.0
dot-product-SUDU_gating,NaN,0.25,16,0.0,NaN,0.0,1220.09,13.74,13.75,1220.09,16.0


In [27]:
layer_dfs["dot-product-DUDU"]

,0
dot-product-DUDU,
gflops,1.990000e+00
utilization,1.000000e+00
cycle,1.280000e+02
energy,0.000000e+00
edp,2.870000e-07
area,0.000000e+00
total_energy,1.046009e+04
total_area,2.243610e+03


In [28]:
layer_dfs["dot-product-SUDU_no-optimization"]

,0
dot-product-SUDU_no-optimization,
total topology energy,35.44
total topology area,1220.09
max topology cycles,16.00
utilization,1.00
cycle,16.00
energy,0.00
area,0.00
total_energy,1220.09
total_area,35.44


In [29]:
layer_dfs["dot-product-SUDU_gating"]

,0
dot-product-SUDU_gating,
total topology energy,13.75
total topology area,1220.09
max topology cycles,16.00
utilization,0.25
cycle,16.00
energy,0.00
area,0.00
total_energy,1220.09
total_area,13.74


In [30]:
df_summary

,gflops,utilization,cycle,energy,edp,area,total_energy,total_area,total topology energy,total topology area,max topology cycles
dot-product-DUDU,1.99,1.00,128,0.0,2.870000e-07,0.0,10460.09,2243.61,NaN,NaN,NaN
dot-product-SCDU,1.98,1.00,48,0.0,1.290000e-07,0.0,33892.08,2686.06,NaN,NaN,NaN
dot-product-SUDU,1.99,1.00,128,0.0,2.870000e-07,0.0,10460.09,2243.61,NaN,NaN,NaN
dot-product-SUDU_no-optimization,NaN,1.00,16,0.0,NaN,0.0,1220.09,35.44,35.44,1220.09,16.0
dot-product-SUDU_gating,NaN,0.25,16,0.0,NaN,0.0,1220.09,13.74,13.75,1220.09,16.0


In [37]:
edp = df_summary["total_energy"] * df_summary["cycle"]
df_summary["EDP (J*cycle)"] = edp

## Save results to LaTeX table

In [38]:
# arch_to_label = {
#     "eyeriss_like": "Eyeriss",
#     "eyeriss_like_w_gating": "w/ Gating",
#     "eyeriss_like_onchip_compression": "w/ Onchip Compression",
#     "eyeriss_like_shen": "w/ Shen",
# }
arch_to_label = {
    "dot-product-DUDU": "DUDU",
    "dot-product-SCDU": "SCDU w/ Onchip Compression",
    "dot-product-SUDU": "SUDU",
    "dot-product-SUDU_no-optimization": "SUDU w/ No Optimization",
    "dot-product-SUDU_gating": "SUDU w/ gating"
}
cols = ["Energy ($\mu\text{J}$)", "Cycles", "EDP (J*cycle)", "Area ($\text{um}^2$)"]

df_output = df_summary.copy()
df_output.index = [arch_to_label[arch] for arch in df_output.index.values]

df_output["Energy ($\mu\text{J}$)"] = df_output["total_energy"]
df_output["Cycles"] = df_output["cycle"].astype(int)
# Convert area in um^2 to mm^2
df_output["Area ($\text{um}^2$)"] = df_output["total_area"]

df_output = df_output[cols]
df_output

,Energy ($\mu\text{J}$),Cycles,EDP (J*cycle),Area ($\text{um}^2$)
DUDU,10460.09,128,1338891.52,2243.61
SCDU w/ Onchip Compression,33892.08,48,1626819.84,2686.06
SUDU,10460.09,128,1338891.52,2243.61
SUDU w/ No Optimization,1220.09,16,19521.44,35.44
SUDU w/ gating,1220.09,16,19521.44,13.74


In [40]:
print(df_output.to_latex(float_format="%.2f"))

\begin{tabular}{lrrrr}
\toprule
 & Energy ($\mu	ext{J}$) & Cycles & EDP (J*cycle) & Area ($	ext{um}^2$) \\
\midrule
DUDU & 10460.09 & 128 & 1338891.52 & 2243.61 \\
SCDU w/ Onchip Compression & 33892.08 & 48 & 1626819.84 & 2686.06 \\
SUDU & 10460.09 & 128 & 1338891.52 & 2243.61 \\
SUDU w/ No Optimization & 1220.09 & 16 & 19521.44 & 35.44 \\
SUDU w/ gating & 1220.09 & 16 & 19521.44 & 13.74 \\
\bottomrule
\end{tabular}

